## Mask R-CNN  (Mask Region-Based Convolutional Neural Network)

## Step 1: Install and Import Required Packages for Computer Vision Tasks
- Install the torchvision package
- Install specific versions of torch and torchvision
- Import necessary packages

**Note:** Install these packages only when using a local machine, not the Simplilearn lab


In [ ]:
pip install torchvision

In [ ]:
pip install torch==1.8.0 torchvision==0.9.0

In [ ]:
from torchvision import transforms as T
import torch
import torchvision
import numpy as np
import random
from PIL import Image
import cv2
import matplotlib.pyplot as plt

## Step 2: Initialize a PreTrained Mask R-CNN Model and Set it to Evaluation Mode
- Initialize a pre-trained Mask R-CNN model
- Set the model to evaluation mode


In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

**Observation:**
- The Mask R-CNN model is initialized, loaded with pretrained weights, and set to evaluation mode.

## Step 3: Define Object Category Names for Mask R-CNN Predictions in COCO Dataset
- Define a list called **COCO_INSTANCE_CATEGORY_NAMES** which represents the names of object categories in the COCO (Common Objects in Context) dataset.
- Each element in the list corresponds to a specific object category identified by a numeric index.

In [ ]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

## Step 4: Extract Predictions from an Image
- Open the image
- Transform the image to a tensor
- Perform predictions using the model
- Extract prediction scores above the threshold
- Extract masks, bounding boxes, and predicted classes
- Filter masks, boxes, and classes based on prediction threshold



In [ ]:
def get_prediction(img_path, threshold):
  img = Image.open(img_path)

  transform = T.Compose([T.ToTensor()])
  img = transform(img)

  pred = model([img])
  pred_score = list(pred[0]['scores'].detach().numpy())

  pred_t = [pred_score.index(x) for x in pred_score if x>threshold][-1]
  masks = (pred[0]['masks']>0.5).squeeze().detach().cpu().numpy()

  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]

  masks = masks[:pred_t+1]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]

  return masks, pred_boxes, pred_class

## Step 5: Add Random Colors to Image Segments: Enhancing Segmented Masks
- Define a list of random colors
- Create empty arrays for each color channel
- Assign random colors to the pixels based on the image values
- Stack the color channels to create a colored mask



In [ ]:
def random_colour_masks(image):
  colours = [[0, 255, 0],[0, 0, 255],[255, 0, 0],[0, 255, 255],[255, 255, 0],[255, 0, 255],[80, 70, 180],[250, 80, 190],[245, 145, 50],[70, 150, 250],[50, 190, 190]]

  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)

  r[image == 1], g[image == 1], b[image == 1] = colours[random.randrange(0,10)]

  coloured_mask = np.stack([r, g, b], axis=2)
  return coloured_mask

## Step 6: Apply the Instance Segmentation API to Visualize Segmented Objects in an Image
- Perform instance segmentation to obtain masks, bounding boxes, and predicted classes
- Read the image and convert it to the RGB color space
- Iterate over the masks and visualize them on the image
- Generate a random color mask for the current segment
- Overlay the color mask on the original image
- Draw a bounding box around the segment
- Add the predicted class label as text near the bounding box
- Display the final annotated image



In [ ]:
def instance_segmentation_api(img_path, threshold=0.5, rect_th=1, text_size=1, text_th=3):
  masks, boxes, pred_cls = get_prediction(img_path, threshold)
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  for i in range(len(masks)):
    rgb_mask = random_colour_masks(masks[i])
    img = cv2.addWeighted(img, 1, rgb_mask, 0.5, 0)
    print(boxes[i][0])
    cv2.rectangle(img, np.asarray(boxes[i][0], dtype=int), np.asarray(boxes[i][1], dtype=int),color=(0, 255, 0), thickness=rect_th)
    cv2.putText(img,pred_cls[i], np.asarray(boxes[i][0], dtype=int), cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)

  plt.figure(figsize=(20,30))
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  plt.show()

In [ ]:
instance_segmentation_api("test2.jpg")

**Observation:**
- The code will display the input image **test2.jpg** with instance segmentations overlaid, bounding boxes around detected objects, and their respective labels (horse) annotated on or near the bounding boxes.

In [ ]:
instance_segmentation_api("cat2.jpg")